<a href="https://colab.research.google.com/github/jeanbouteiller-ds/tennis_prediction/blob/main/Table_initialization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import requests
import io
import importlib.util
import sys
import nbformat
import pandas as pd

# Define the URL of the nb1 notebook on GitHub
github_url = "https://raw.githubusercontent.com/jeanbouteiller-ds/tennis_prediction/main/functions/functions_fixed_table.ipynb"

# Download the notebook as a raw .ipynb file
response = requests.get(github_url)

if response.status_code == 200:
  notebook_content = response.text

  # Parse the notebook content
  notebook = nbformat.reads(notebook_content, as_version=4)

  # Now you can execute the cells in the notebook
  for cell in notebook.cells:
      if cell.cell_type == 'code':
          exec(cell.source)

# Fixed Table initialisation

In [53]:
start_date = datetime(1990, 1, 1)
end_date = datetime(2023, 10, 15)

list_all_names,list_all_urls_final=create_initial_list(start_date,end_date)

# # Create a new file with content
file_name_player_names = "data/list_all_players_names.txt"  # The name of the new file
file_name_urls = "data/list_all_urls.txt"  # The name of the new file
list_all_names_str=str(list_all_names)
list_all_urls_str=str(list_all_urls_final)

add_file_github(file_name_player_names,list_all_names_str)
add_file_github(file_name_urls,list_all_urls_str)

In [47]:
list_player_names=get_file_content('data/list_all_players_names.txt')
list_urls=get_file_content('data/list_all_urls.txt')
df_fixed=create_new_fixed_table('data/df_player_fixed.csv',list_player_names,list_urls,write_csv=True)

# Tournament Table

Note That this code can be re-run but we don't want to recreate a new workflow fort that

In [48]:
tournament_list=[]
list_year=[1990 + k for k in range(35)]
for year in list_year:
  for element in html_elements_from_url("https://www.atptour.com/en/scores/results-archive?year="+str(year),"tr",["tourney-result"])["tourney-result"]:
    tournament_dict={}
    tournament_dict['url']=element.find('a').get("href")
    tournament_dict['tournament_name']=element.find('span',class_="tourney-location").text.strip()
    tournament_dict['tournament_date']=element.find('span',class_="tourney-dates").text.strip()
    tournament_dict['number_players']=element.find_all('td',class_="tourney-details")[0].find_all('span',class_="item-value")[0].text.strip().strip()
    tournament_dict['indoor_outdoor']=element.find_all('td',class_="tourney-details")[1].find_all('div',class_="item-details")[0].text.split('\r\n')[1].strip()
    tournament_dict['surface']=element.find_all('td',class_="tourney-details")[1].find_all('span',class_="item-value")[0].text.strip()
    tournament_dict['prize_money']=element.find_all('td',class_="tourney-details fin-commit")[0].text.strip()
    tournament_list.append(tournament_dict)

add_file_github("data/df_tournaments.csv",pd.DataFrame(tournament_list).to_csv(index=False))


# Matchs table

In [68]:
import pandas as pd
from tqdm import tqdm
import re

# df_all_matchs=pd.DataFrame(columns=get_file_content("data/df_all_matchs.csv").columns)
df_all_matchs=get_file_content("data/df_all_matchs.csv")
df_tournaments=get_file_content("data/df_tournaments.csv")
df_tournaments=df_tournaments[(~(df_tournaments['tournament_name']+df_tournaments['tournament_date']).isin((df_all_matchs['tournament_name']+df_all_matchs['tournament_date']).unique()))]

list_matchs=[]
nb_batches=10
for k in range(nb_batches):
  start=(k)/nb_batches
  end=(k+1)/nb_batches
  start_rows=int(start*len(df_tournaments))
  end_rows=int(end*len(df_tournaments))

  for index,row in tqdm(df_tournaments[start_rows:end_rows].iterrows(), total=end_rows-start_rows):
    print(row)
    list_matchs+=find_matchs(row)

  add_file_github("data/df_all_matchs.csv",pd.concat([df_all_matchs,pd.DataFrame(list_matchs)]).to_csv(index=False))

# Table Ranking

In [124]:
start_date = datetime(1990, 1, 1)
end_date = datetime(2023, 10, 15)

list_urls=find_list_weekly_ranking_url(start_date, end_date,nb_players_ranking=300)
# df_all_ranking=pd.DataFrame(columns=['player_name','ranking','date'])
df_all_ranking=get_file_content("data/df_all_ranking.csv")

list_urls=[k for k in list_urls if k.split("Date=")[1] not in df_all_ranking["date"].values]
print(len(list_urls))

for week in range(len(list_urls)):
  weekly_ranking_url=list_urls[week]
  df_all_ranking=pd.concat([df_all_ranking,get_ranking(weekly_ranking_url)]).reset_index(drop=True).drop_duplicates()

  if week%20==0:
    print(weekly_ranking_url)
    add_file_github("data/df_all_ranking.csv",df_all_ranking.to_csv(index=False))

add_file_github("data/df_all_ranking.csv",df_all_ranking.to_csv(index=False))


In [127]:
get_file_content("data/df_all_ranking.csv")

<string>:44: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.


,player_name,ranking,date
0,Ivan Lendl,1,1990-01-01
1,Boris Becker,2,1990-01-01
2,Stefan Edberg,3,1990-01-01
3,Brad Gilbert,4,1990-01-01
4,John McEnroe,5,1990-01-01
...,...,...,...
496599,Skander Mansouri,296,2023-07-31
496600,Jules Marie,297,2023-07-31
496601,Matheus Pucinelli De Almeida,298,2023-07-31
496602,Nino Serdarusic,299,2023-07-31


# Betting Data

In [150]:
# URL of the online-hosted Excel dataset
columns_selected=['Location','Tournament','Date','Winner','Loser']
df_all_betting=pd.DataFrame(columns=columns_selected)
for year in [2001+k for k in range(23)]:
  excel_url = "http://www.tennis-data.co.uk/"+str(year)+"/"+str(year)+".xlsx"

  # Read the Excel dataset into a Pandas DataFrame
  df = pd.read_excel(excel_url)

  odds_columns=[k for k in df.columns if k[-1]=='L' or k[-1]=='W']
  columns_selected_year=columns_selected+odds_columns
  # print(df.columns)
  df=df[columns_selected_year]

  # Display the first few rows of the DataFrame to verify the data
  df_all_betting=pd.concat([df_all_betting,df]).reset_index(drop=True)

add_file_github("data/df_all_betting.csv",df_all_betting.to_csv(index=False))


/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
